### Importando Bibliotecas

In [137]:
from io import StringIO
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.by import By


from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Date, DECIMAL
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import update


### Usando Selenium para extrair as informações

In [138]:
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless")
browser = webdriver.Chrome(options=chrome_options)
browser.get("https://www.bcb.gov.br/controleinflacao/historicotaxasjuros")

In [139]:
try:

    wait = WebDriverWait(browser, 50)
    wait.until(
        lambda browser: browser.execute_script("return document.readyState") == "complete"
    )

    xpath = wait.until(
        EC.presence_of_element_located((By.XPATH, '/html/body/app-root/app-root/div/div/main/dynamic-comp/div/div/bcb-histtaxajuros/div[1]/table'))
    )

    table_html = xpath.get_attribute('outerHTML')
    browser.quit()

    df = pd.read_html(StringIO(table_html))[0]

except Exception as e:
    print(f"Ocorreu um erro: {e}")

finally:
    if 'browser' in locals() or 'browser' in globals():
        browser.quit()

In [140]:
df

Reunião                       Período de vigência  \
         nº        data viés      Período de vigência   
0      262º  08/05/2024  NaN             09/05/2024 -   
1      261º  20/03/2024  NaN  21/03/2024 - 08/05/2024   
2      260º  31/01/2024  NaN  01/02/2024 - 20/03/2024   
3      259º  13/12/2023  NaN  14/12/2023 - 31/01/2024   
4      258º  01/11/2023  NaN  03/11/2023 - 13/12/2023   
..      ...         ...  ...                      ...   
265      5º  23/10/1996  NaN  01/11/1996 - 30/11/1996   
266      4º  23/09/1996  NaN  01/10/1996 - 31/10/1996   
267      3º  21/08/1996  NaN  01/09/1996 - 30/09/1996   
268      2º  30/07/1996  NaN  01/08/1996 - 31/08/1996   
269      1º  26/06/1996  NaN  01/07/1996 - 31/07/1996   

    Meta SELIC % a.a. (1)(6) TBAN % a.m. (2)(6) Taxa SELIC            
    Meta SELIC % a.a. (1)(6) TBAN % a.m. (2)(6)      % (3) % a.a.(4)  
0                       1050                NaN        NaN       NaN  
1                       1075                NaN      133.0    1065.0  
2                       1125                NaN      139.0    1115.0  
3                       1175                NaN      145.0    1165.0  
4                       1225                NaN      128.0    1215.0  
..                       ...                ...        ...       ...  
265                      178              190.0      180.0    2527.0  
266                      182              193.0      186.0    2348.0  
267                      188                NaN      190.0    2540.0  
268                      190                NaN      197.0    2501.0  
269                      190                NaN      193.0    2328.0  

[270 rows x 8 columns]

### Transformando os dados

In [141]:
df.columns = df.columns.get_level_values(1)

In [142]:
rename_columns = [
        "num_reuniao",
        "data_reuniao",
        "vies_reuniao",
        "periodo",
        "meta_selic",
        "tban",
        "taxa_selic_porcentagem",
        "taxa_selic_a_a",
    ]
df.columns = rename_columns

In [143]:
df

,num_reuniao,data_reuniao,vies_reuniao,periodo,meta_selic,tban,taxa_selic_porcentagem,taxa_selic_a_a
0,262º,08/05/2024,NaN,09/05/2024 -,1050,NaN,NaN,NaN
1,261º,20/03/2024,NaN,21/03/2024 - 08/05/2024,1075,NaN,133.0,1065.0
2,260º,31/01/2024,NaN,01/02/2024 - 20/03/2024,1125,NaN,139.0,1115.0
3,259º,13/12/2023,NaN,14/12/2023 - 31/01/2024,1175,NaN,145.0,1165.0
4,258º,01/11/2023,NaN,03/11/2023 - 13/12/2023,1225,NaN,128.0,1215.0
...,...,...,...,...,...,...,...,...
265,5º,23/10/1996,NaN,01/11/1996 - 30/11/1996,178,190.0,180.0,2527.0
266,4º,23/09/1996,NaN,01/10/1996 - 31/10/1996,182,193.0,186.0,2348.0
267,3º,21/08/1996,NaN,01/09/1996 - 30/09/1996,188,NaN,190.0,2540.0
268,2º,30/07/1996,NaN,01/08/1996 - 31/08/1996,190,NaN,197.0,2501.0


In [144]:
df.dtypes

num_reuniao                object
data_reuniao               object
vies_reuniao               object
periodo                    object
meta_selic                  int64
tban                      float64
taxa_selic_porcentagem    float64
taxa_selic_a_a            float64
dtype: object

In [145]:
# Limpando e separando dados
df["num_reuniao"] = df["num_reuniao"].str.replace(r'\D+', '', regex=True)
df[["inicio_vigencia", "fim_vigencia"]] = df["periodo"].str.extract(r"([\d/]+)\s*-\s*(\s*[\d/]*)")
df.drop('periodo', axis=1, inplace=True)

In [146]:
df['num_reuniao'] = df['num_reuniao'].astype(int)

df['meta_selic'] = df['meta_selic'].astype(float) / 100
df['tban'] = df['tban'].astype(float) / 100
df['taxa_selic_porcentagem'] = df['taxa_selic_porcentagem'].astype(float) / 100
df['taxa_selic_a_a'] = df['taxa_selic_a_a'].astype(float) / 100

df['data_reuniao'] = pd.to_datetime(df['data_reuniao'], format='%d/%m/%Y').dt.strftime('%d%m%Y').fillna('01011900')
df['inicio_vigencia'] = pd.to_datetime(df['inicio_vigencia'], format='%d/%m/%Y').dt.strftime('%d%m%Y').fillna('01011900')
df['fim_vigencia'] = pd.to_datetime(df['fim_vigencia'], format='%d/%m/%Y').dt.strftime('%d%m%Y').fillna('01011900')

df.replace(np.nan, None, inplace=True)


In [156]:
df.dtypes

num_reuniao                 int32
data_reuniao               object
vies_reuniao               object
meta_selic                float64
tban                       object
taxa_selic_porcentagem     object
taxa_selic_a_a             object
inicio_vigencia            object
fim_vigencia               object
dtype: object

In [157]:
df

,num_reuniao,data_reuniao,vies_reuniao,meta_selic,tban,taxa_selic_porcentagem,taxa_selic_a_a,inicio_vigencia,fim_vigencia
0,262,08052024,None,10.50,None,None,None,09052024,01011900
1,261,20032024,None,10.75,None,1.33,10.65,21032024,08052024
2,260,31012024,None,11.25,None,1.39,11.15,01022024,20032024
3,259,13122023,None,11.75,None,1.45,11.65,14122023,31012024
4,258,01112023,None,12.25,None,1.28,12.15,03112023,13122023
...,...,...,...,...,...,...,...,...,...
265,5,23101996,None,1.78,1.9,1.8,25.27,01111996,30111996
266,4,23091996,None,1.82,1.93,1.86,23.48,01101996,31101996
267,3,21081996,None,1.88,None,1.9,25.4,01091996,30091996
268,2,30071996,None,1.90,None,1.97,25.01,01081996,31081996


### Usando SQL Alchemy ORM para inserir no Banco destino

In [164]:
Base = declarative_base()

class Reuniao(Base):
    __tablename__ = 'historico_taxas_juros'

    num_reuniao = Column(Integer, primary_key=True)
    data_reuniao = Column(String(8), nullable=True)
    vies_reuniao = Column(String(50),nullable=True)
    meta_selic = Column(DECIMAL(10, 2), nullable=True)
    tban = Column(DECIMAL(10, 2), nullable=True)
    taxa_selic_porcentagem = Column(DECIMAL(10, 2), nullable=True)
    taxa_selic_a_a = Column(DECIMAL(10, 2), nullable=True)
    inicio_vigencia = Column(String(8), nullable=True)
    fim_vigencia = Column(String(8), nullable=True)


C:\Users\tarmi\AppData\Local\Temp\ipykernel_25648\1978180080.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [171]:
class DBEngine:
    def __init__(self, db_type, db_local, db_user, db_pass, db_name):
        self.db_type = db_type
        self.db_local = db_local
        self.db_user = db_user
        self.db_pass = db_pass
        self.db_name = db_name
        self.engine = self.create_engine()
        self.session = sessionmaker(bind=self.engine)

    def create_engine(self):
        if self.db_type == 'postgresql':
            # Criação da engine para PostgreSQL
            db_url = f'postgresql://{self.db_user}:{self.db_pass}@{self.db_local}/{self.db_name}'
        elif self.db_type == 'sqlserver':
            # Criação da engine para SQL Server
            db_url = f'mssql+pyodbc://{self.db_user}:{self.db_pass}@{self.db_local}/{self.db_name}?driver=ODBC+Driver+17+for+SQL+Server'
        else:
            raise ValueError('Tipo de banco de dados não suportado')

        engine = create_engine(db_url)
        return engine

    def get_session(self):
        return self.session()


In [172]:
engine = DBEngine("postgresql", "localhost:5437", "admin", "admin", "bcb")
session = engine.get_session()

In [173]:
def load_data(session,df):
    for _, row in df.iterrows():
        num_reuniao = row['num_reuniao']
        existing_reuniao = session.query(Reuniao).filter_by(num_reuniao=num_reuniao).first()

        if existing_reuniao:
            # Verifica se algum campo precisa ser atualizado
            changes = {key: value for key, value in row.items() if getattr(existing_reuniao, key) != value}
            if changes:
                session.execute(
                    update(Reuniao)
                    .where(Reuniao.num_reuniao == num_reuniao)
                    .values(**changes)
                )
        else:
            # Cria um novo registro se não existir
            new_reuniao = Reuniao(**row)
            session.add(new_reuniao)

    session.commit()
    session.close()

In [174]:
load_data(session, df)